### Import API Keys

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

### Import Tools

In [2]:
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun 
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory
from langchain.tools import ShellTool
from langchain.agents import load_tools



# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()
#Shell Tool
shell_tool = ShellTool()
requests_tools = load_tools(["requests_all"])

#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()

#Python REPL Class
python_repl = PythonREPL()

shell_tool = Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
)


#DuckDuckGo Tool
duckduck_tool=Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
)

#Python REPL Agent Tool Class
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)
toolkit = FileManagementToolkit(root_dir=str(working_directory.name)) # If you don't provide a root_dir, operations will default to the current working directory
directory_tools = toolkit.get_tools()

In [3]:

tools = [
    WriteFileTool(),
    ReadFileTool(),
    repl_tool,
    duckduck_tool, 
    #directory_tools, 
    shell_tool, 
    #requests_tool
]

In [4]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [5]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [6]:
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

In [7]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Sameul",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    memory=vectorstore.as_retriever()
)
# Set verbose to be true
#agent.chain.verbose = True

In [8]:
agent.run(["Create a folder called 'Proteus'. Make a HubSpot module that will get a number's country code and attach to a phone number based on the country code on their contact details"])

{
    "thoughts": {
        "text": "I think I should start by creating the 'Proteus' folder and then move on to creating the HubSpot module. I will use the 'write_file' command to create the folder.",
        "reasoning": "Creating the folder will provide a place to store the module and any other files related to the project. It is important to have a well-organized file structure to make it easier to find and manage files.",
        "plan": "- Use the 'write_file' command to create the 'Proteus' folder\n- Create the HubSpot module and save it in the 'Proteus' folder",
        "criticism": "I need to make sure that I am using the correct file path when creating the folder. I also need to make sure that the folder is created successfully before moving on to the next step.",
        "speak": "I will start by creating the 'Proteus' folder using the 'write_file' command."
    },
    "command": {
        "name": "write_file",
        "args": {
            "file_path": "Proteus",
          